<a href="https://colab.research.google.com/github/hyeonseong0917/NLP_study/blob/main/Korean_Word_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
dataset = dataset.data

In [ ]:
dataset[0]

In [ ]:
print(f"데이터셋 총 개수 : {len(dataset)}")

In [ ]:
news_df = pd.DataFrame({'document':dataset})
news_df

In [ ]:
# 먼저 데이터셋 내에 결측값이 있는지 확인합니다
news_df.replace("", float("NaN"), inplace=True)
print(news_df.isnull().values.any())

In [ ]:
news_df = news_df.dropna().reset_index(drop=True)
print(f"필터링된 데이터셋 총 개수 : {len(news_df)}")

In [ ]:
news_df

In [ ]:
# 열을 기준으로 중복제거
processed_news_df = news_df.drop_duplicates(['document']).reset_index(drop=True)
processed_news_df

In [ ]:
len(processed_news_df.iloc[0][0])

In [ ]:
processed_news_df.iloc[0][0]

In [ ]:
# 데이터셋 내 특수 문자를 제거합니다.
processed_news_df['document'] = processed_news_df['document'].str.replace("[^a-zA-Z]", " ")
processed_news_df

In [ ]:
# 문서 내 길이가 너무 짧은 단어를 제거합니다. (단어의 길이가 2 이하)
processed_news_df['document'] = processed_news_df['document'].apply(lambda x: ' '.join([token for token in x.split() if len(token) > 2]))
processed_news_df

In [ ]:
# 전체 길이가 200 이하이거나 전체 단어 개수가 5개 이하인 데이터를 필터링합니다.
processed_news_df = processed_news_df[processed_news_df.document.apply(lambda x: len(str(x)) <= 200 and len(str(x).split()) > 5)].reset_index(drop=True)
processed_news_df

In [ ]:
# 전체 단어에 대한 소문자 변환 (정규화)
processed_news_df['document'] = processed_news_df['document'].apply(lambda x: x.lower())
processed_news_df

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
stop_words = stopwords.words('english')

print(len(stop_words))
print(stop_words[:10])

In [ ]:
# 데이터들의 불용어를 제외하여 띄어쓰기 단위로 문장을 분리합니다.
tokenized_doc = processed_news_df['document'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [s_word for s_word in x if s_word not in stop_words])
tokenized_doc

In [ ]:
tokenized_doc = tokenized_doc.to_list()
print(len(tokenized_doc))

In [ ]:
print(f"최종 학습 데이터셋 수 : {len(tokenized_doc)}")

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_doc)

In [ ]:
word2idx = tokenizer.word_index
idx2word = {value : key for key, value in word2idx.items()}
encoded = tokenizer.texts_to_sequences(tokenized_doc)

In [ ]:
vocab_size = len(word2idx) + 1 
print(f"단어 사전의 크기 : {vocab_size}")

In [ ]:
print(encoded[0])

In [ ]:
from tensorflow.keras.preprocessing.sequence import skipgrams

In [ ]:
# 네거티브 샘플링
skip_grams = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded[:5]]
print(f"전체 샘플 수 : {len(skip_grams)}")

In [ ]:
# 첫번째 샘플인 skip_grams[0] 내 skipgrams로 형성된 데이터셋 확인
pairs, labels = skip_grams[0][0], skip_grams[0][1]

In [ ]:
print(f"first 3 pairs: {pairs[:3]}")
print(f"first 3 labels: {labels[:3]}")

In [ ]:
# 첫번째 뉴스그룹 샘플에 대해서 생긴 pairs와 labels의 개수
print(len(pairs))
print(len(labels))

In [ ]:
for i in range(5):
  print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
    idx2word[pairs[i][0]], pairs[i][0], 
    idx2word[pairs[i][1]], pairs[i][1], 
    labels[i])
  )

In [ ]:
training_dataset = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded[:1000]]
print(training_dataset[0][0])

In [ ]:
print(len(training_dataset))

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input
from tensorflow.keras.layers import Dot
from tensorflow.keras.utils import plot_model
from IPython.display import SVG

In [ ]:
embedding_dim = 100

# 중심 단어를 위한 임베딩 테이블
w_inputs = Input(shape=(1, ), dtype='int32')
word_embedding = Embedding(vocab_size, embedding_dim)(w_inputs)

# 주변 단어를 위한 임베딩 테이블
c_inputs = Input(shape=(1, ), dtype='int32')
context_embedding  = Embedding(vocab_size, embedding_dim)(c_inputs)

In [ ]:
dot_product = Dot(axes=2)([word_embedding, context_embedding])

dot_product = Reshape((1,), input_shape=(1, 1))(dot_product)
print(dot_product)

In [ ]:
dot_product = Dot(axes=2)([word_embedding, context_embedding])
print(dot_product)
dot_product = Reshape((1,), input_shape=(1, 1))(dot_product)
print(dot_product)
output = Activation('sigmoid')(dot_product)
# print(output[0][0])
import keras.backend as K
import numpy as np
import tensorflow as tf
# tf.config.run_functions_eagerly(True)
# output.numpy()
model = Model(inputs=[w_inputs, c_inputs], outputs=output)
print(model.output)
# model.summary()
# model.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:
dot_product = Dot(axes=2)([word_embedding, context_embedding])
dot_product = Reshape((1,), input_shape=(1, 1))(dot_product)
output = Activation('sigmoid')(dot_product)

model = Model(inputs=[w_inputs, c_inputs], outputs=output)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam')
plot_model(model, to_file='model3.png', show_shapes=True, show_layer_names=True, rankdir='TB')

In [ ]:
for epoch in range(10):
  loss = 0
  for _, elem in enumerate(skip_grams):
    first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
    second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
    labels = np.array(elem[1], dtype='int32')
    X = [first_elem, second_elem]
    Y = labels
    loss += model.train_on_batch(X,Y)  
  print('Epoch :',epoch + 1, 'Loss :',loss)

In [ ]:
import gensim

In [ ]:
f = open('vectors.txt' ,'w')
f.write('{} {}\n'.format(vocab_size-1, embedding_dim))
vectors = model.get_weights()[0]
for word, i in tokenizer.word_index.items():
  f.write('{} {}\n'.format(word, ' '.join(map(str, list(vectors[i, :])))))
f.close()

In [ ]:
# 모델 로드
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)

In [ ]:
k=w2v.most_similar(negative=['apple'])
k
# k[0]

In [ ]:
data=[
    {
        "교과목": "사회",
        "텍스트": "자정 능력: 미생물이 오염 물질을 분해하여 원래의 깨끗한 상태로 되돌리는 능력이다."
    },
    {
        "교과목": "사회",
        "텍스트": "남획으로 줄어드는 어족 자원: 참치는 현대식 어업을 통해 대량으로 잡히는 생선 중 하나이다.\n선망 어선은 길이 2km, 깊이 200m정도인 대형 그물을 이용하여 참치뿐만 아니라 멸종 위기에 처한 상어, 가오리, 고래, 바다거북 등을 가리지 않고 잡아들인다.\n이른바 '혼획'이라고 하는 이 어업 방식으로 매년 많은 종류의 물고기가 멸종되고 있다.\n외래종의 유입으로 사라지는 토착종: 외래종의 유입은 자연스럽게 발생할 수 있으나, 인위적인 목적으로 이루어지면 심각한 위험을 불러올 수 있다.\n무분별하게 도입해 방사한 황소개구리에 의해 여러 종류의 민물고기가 멸종 위기에 처해 있는 상황과 나일 농어의 유입으로 200여 종의 물고기가 사라진 아프리카 빅토리아호가 대표적인 사례이다.\n수질 오염에 따른 수상 생물 종의 감소: 농업 생산량을 늘리고자 사용하는 질소와 인산 성분 비료는 장기적으로 토양뿐만 아니라 하천과 바다를 오염시킨다.\n1860년 이후 바다로 흘러들어 간 질소의 양은 2배로 늘어났다.\n바다에 녹아든 질소는 조류 (藻類)의 이상 번식을 일으켜 산소를 고갈시키고 수상 생물을 떼죽음으로 내몬다."
    },
    {
        "교과목": "사회",
        "텍스트": "인간 개발 지수 (HDI): 각국의 교육 수준과 국민 소득, 기대 수명 등을 기준으로 발전 정도를 평가하는 지수이다."
    },
    {
        "교과목": "사회",
        "텍스트": "세계 각 지역은 부존 자원 정도, 산업화 시기, 역사적 배경 등의 영향으로 발전 수준에 차이가 난다.\n일반적으로 저개발 지역은 발전 지역보다 소득 수준이 낮으며 학교나 의료 시설 등이 부족하고 주거 환경도 열악하여 삶의 질이 낮다.\n저개발 지역들은 빈곤에 따른 여러 사회 문제를 해결하고 경제 발전을 이루어 삶의 질을 높이고자 적극적으로 노력하고 있다.\n사회 기반 시설을 확충하기 위해 외국 자본의 유치를 적극적으로 추진하고 있으며, 선진 기술의 도입으로 산업 부분에서의 생산성을 높이고 있다.\n최근에는 세계화에 따라 국가 사이의 경쟁이 심해지면서 이에 대응하려는 저개발 지역들이 경제 협력 체제를 만들고 있다.\n실제로 2009 ~ 2013년 동안 저개발 지역 국가들의 연평균 경제 성장률은 선진국들보다 높게 나타난다."
    },
    {
        "교과목": "사회",
        "텍스트": "경제 협력 체제는 여러 국가가 세계화에 대응하고 경쟁력을 높이고자 구성하는 것이다.\n특히, 단일 국가의 능력으로 선진국들의 자본, 기술과 경쟁하기 어려운 저개발 국가들은 협력 체제를 구축하여 공동으로 자원을 개발하여 수출하고, 자국의 이익에 부정적인 영향을 미치는 국가들에 함께 대응하는 노력이 이루어지고 있다.\n경제 협력 체제 회원국 간에는 수출 시 부과하는 세금을 낮추거나 없애 교류를 확대하고 이를 바탕으로 상호 보완적인 경제 발전을 추진할 수 있다."
    },
    {
        "교과목": "사회",
        "텍스트": "국제 사회는 세계 각 지역의 불평등을 완화하고자 다양한 방면으로 노력하고 있다.\n국제 연합을 중심으로 각국 정부는 공적 개발 원조를 통한 경제적 지원뿐만 아니라 스포츠, 문화 등 다양한 분야에서 협력하고 있다.\n민간 및 개인 차원에서의 협력도 활발히 이루어지고 있다.\n특히 비정부 기구 (NGO)는 저개발 지역의 어려운 현실을 시민들에게 알리고 이들 지역 주민들을 돕는 다양한 기회를 제공한다."
    },
    {
        "교과목": "사회",
        "텍스트": "공적 개발 원조: 선진국의 정부 또는 공공 기관이 저개발국의 경제 사회 발전과 복지 증진을 주목적으로 자금이나 기술을 지원하는 제도이다."
    },
    {
        "교과목": "사회",
        "텍스트": "비정부 기구 (Non-Goverment Organization): 권력이나 이윤을 추구하지 않고 조직된 자발적인 시민 단체이다."
    },
]
texts=[]
for i in range(len(data)):
  texts.append(data[i]["텍스트"].replace("\n",""))
print(texts)  


In [ ]:
kor_tokenized_doc = [text.split() for text in texts]
print(len(kor_tokenized_doc))

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(kor_tokenized_doc)

In [ ]:
word2idx = tokenizer.word_index
idx2word = {value : key for key, value in word2idx.items()}
encoded = tokenizer.texts_to_sequences(kor_tokenized_doc)

In [ ]:
vocab_size = len(word2idx) + 1 
print(f"단어 사전의 크기 : {vocab_size}")

In [ ]:
print(encoded[0])

In [ ]:
from tensorflow.keras.preprocessing.sequence import skipgrams

In [ ]:
# 네거티브 샘플링
skip_grams = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded[:5]]
print(f"전체 샘플 수 : {len(skip_grams)}")

In [ ]:
# 첫번째 샘플인 skip_grams[0] 내 skipgrams로 형성된 데이터셋 확인
pairs, labels = skip_grams[0][0], skip_grams[0][1]

In [ ]:
# 첫번째 뉴스그룹 샘플에 대해서 생긴 pairs와 labels의 개수
print(len(pairs))
print(len(labels))

In [ ]:
print(pairs)

In [ ]:
for i in range(5):
  print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
    idx2word[pairs[i][0]], pairs[i][0], 
    idx2word[pairs[i][1]], pairs[i][1], 
    labels[i])
  )

In [ ]:
training_dataset = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded]

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input
from tensorflow.keras.layers import Dot
from tensorflow.keras.utils import plot_model
from IPython.display import SVG

In [ ]:
embedding_dim = 100

# 중심 단어를 위한 임베딩 테이블
w_inputs = Input(shape=(1, ), dtype='int32')
word_embedding = Embedding(vocab_size, embedding_dim)(w_inputs)

# 주변 단어를 위한 임베딩 테이블
c_inputs = Input(shape=(1, ), dtype='int32')
context_embedding  = Embedding(vocab_size, embedding_dim)(c_inputs)

In [ ]:
dot_product = Dot(axes=2)([word_embedding, context_embedding])
dot_product = Reshape((1,), input_shape=(1, 1))(dot_product)
output = Activation('sigmoid')(dot_product)

model = Model(inputs=[w_inputs, c_inputs], outputs=output)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam')
plot_model(model, to_file='model3.png', show_shapes=True, show_layer_names=True, rankdir='TB')

In [ ]:
for epoch in range(10):
  loss = 0
  for _, elem in enumerate(skip_grams):
    first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
    second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
    labels = np.array(elem[1], dtype='int32')
    X = [first_elem, second_elem]
    Y = labels
    loss += model.train_on_batch(X,Y)  
  print('Epoch :',epoch + 1, 'Loss :',loss)

In [ ]:
f = open('kor_vectors.txt' ,'w')
f.write('{} {}\n'.format(vocab_size-1, embedding_dim))
vectors = model.get_weights()[0]
for word, i in tokenizer.word_index.items():
  f.write('{} {}\n'.format(word, ' '.join(map(str, list(vectors[i, :])))))
f.close()

In [ ]:
# 모델 로드
import re
w2v = gensim.models.KeyedVectors.load_word2vec_format('./kor_vectors.txt', binary=False)
for i in range(len(texts)):
  # print(texts[i])
  cur_text=texts[i]
  
  # print(type(cur_text))
  
  cur_list=re.split(" |\.",cur_text)
  # print(re.split(" |\.",cur_text))
  # for j in range(len(cur_list)):
  #   cur_word=cur_list[j]
  #   if cur_word in w2v:
  #     k=w2v.most_similar(negative=[cur_word])
  #     print(k)

# if "선진국" in w2v:
#   print("hello")

In [ ]:
k=w2v.most_similar(negative=['선진국'])
print(k)
# print(t)
# for i in len(texts):
